<a href="https://colab.research.google.com/github/amien1410/colab-notebooks/blob/main/Colab_Pyspark_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install pyspark
!pip install pyspark

# initiate pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

# download sample
!wget -O Mall_Customers.csv "https://raw.githubusercontent.com/Shahadat9886/Mall_Customers.csv/refs/heads/main/Thu%20H%C3%A0%20Tr%E1%BA%A7n%20-%20Mall_Customers.csv"

# load csv
df = spark.read.csv("/content/Mall_Customers.csv", header=True, inferSchema=True)
df

--2025-06-10 05:59:32--  https://raw.githubusercontent.com/Shahadat9886/Mall_Customers.csv/refs/heads/main/Thu%20H%C3%A0%20Tr%E1%BA%A7n%20-%20Mall_Customers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4286 (4.2K) [text/plain]
Saving to: ‘Mall_Customers.csv’

Mall_Customers.csv  100%[===================>]   4.19K  --.-KB/s    in 0s      

2025-06-10 05:59:33 (49.8 MB/s) - ‘Mall_Customers.csv’ saved [4286/4286]



DataFrame[CustomerID: int, Genre: string, Age: int, Annual Income (k$): int, Spending Score (1-100): int]

In [2]:
# use spark.sql
df2 = spark.sql("select 'spark' as hello ")
df2.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [11]:
# !wget -O movies.csv "https://raw.githubusercontent.com/Apress/applied-data-science-using-pyspark/refs/heads/main/Ch02/Chapter2_Data/movie_data_part1.csv"
df = spark.read.csv("movies.csv", header=True, sep='|', inferSchema=False)
df.show(10)

+---------------------+------+-----+-----------------+--------------------+--------------------+----------+--------------------+--------------------+------------+-------+-------+--------+--------------------+--------------------+------------+
|belongs_to_collection|budget|   id|original_language|      original_title|            overview|popularity|production_companies|production_countries|release_date|revenue|runtime|  status|             tagline|               title|vote_average|
+---------------------+------+-----+-----------------+--------------------+--------------------+----------+--------------------+--------------------+------------+-------+-------+--------+--------------------+--------------------+------------+
|                 NULL|     0|43000|               fr|  Le Caporal épinglé|The story serves ...|     2.503|[{'id': 16059, 'l...|[{'iso_3166_1': '...|  1962-05-23|      0|     90|Released|                NULL|The Elusive Corporal|         5.9|
|                 NULL|     

In [5]:
# Importing the SparkContext module from PySpark
from pyspark import SparkContext

# Creating a SparkContext object
sc = SparkContext.getOrCreate()

# Importing the random module
import random
NUM_SAMPLES = 100000000 # Define the samples

# Function to check if a point lies inside
def inside(p):
  x, y = random.random(), random.random()
  return x*x + y*y < 1

# parallelize the computation
count = sc.parallelize(range(0, NUM_SAMPLES)).filter(inside).count()

# calculate the estimate value of pi
pi = 4 * count / NUM_SAMPLES
print("Pi is roughly", pi)

Pi is roughly 3.14192564


In [12]:
df.printSchema()

root
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- vote_average: string (nullable = true)



In [13]:
df.dtypes

[('belongs_to_collection', 'string'),
 ('budget', 'string'),
 ('id', 'string'),
 ('original_language', 'string'),
 ('original_title', 'string'),
 ('overview', 'string'),
 ('popularity', 'string'),
 ('production_companies', 'string'),
 ('production_countries', 'string'),
 ('release_date', 'string'),
 ('revenue', 'string'),
 ('runtime', 'string'),
 ('status', 'string'),
 ('tagline', 'string'),
 ('title', 'string'),
 ('vote_average', 'string')]

In [14]:
print('The total number of records in the movie dataset is '+str(df.count()))

The total number of records in the movie dataset is 43998


In [15]:
#@title Subset Columns

# Defining a list to subset the required columns
select_columns=['id', 'budget', 'popularity', 'release_date', 'revenue','title']

# Subsetting the required columns from the DataFrame
df=df.select(*select_columns)

# The following command displays the data; by default it shows top 20 rows
df.show()

+-----+-------+------------------+------------+-------+--------------------+
|   id| budget|        popularity|release_date|revenue|               title|
+-----+-------+------------------+------------+-------+--------------------+
|43000|      0|             2.503|  1962-05-23|      0|The Elusive Corporal|
|43001|      0|              5.51|  1962-11-12|      0|  Sundays and Cybele|
|43002|      0|              5.62|  1962-05-24|      0|Lonely Are the Brave|
|43003|      0|             7.159|  1975-03-12|      0|          F for Fake|
|43004| 500000|             3.988|  1962-10-09|      0|Long Day's Journe...|
|43006|      0|             3.194|  1962-03-09|      0|           My Geisha|
|43007|      0|             2.689|  1962-10-31|      0|Period of Adjustment|
|43008|      0|             6.537|  1959-03-13|      0|    The Hanging Tree|
|43010|      0|             4.297|  1962-01-01|      0|Sherlock Holmes a...|
|43011|      0|             4.417|  1962-01-01|      0|  Sodom and Gomorrah|

In [16]:
#@title Missing Values

from pyspark.sql.functions import *

df = spark.read.csv("movies.csv", header=True, sep='|', inferSchema=False)
df.filter((df['popularity']=='')|df['popularity'].isNull()|isnan(df['popularity'])).count()

215

In [17]:
df.select([count(when((col(c)=='') | col(c).isNull() |isnan(c), c)).alias(c)
for c in df.columns]).show()

+---------------------+------+---+-----------------+--------------+--------+----------+--------------------+--------------------+------------+-------+-------+------+-------+-----+------------+
|belongs_to_collection|budget| id|original_language|original_title|overview|popularity|production_companies|production_countries|release_date|revenue|runtime|status|tagline|title|vote_average|
+---------------------+------+---+-----------------+--------------+--------+----------+--------------------+--------------------+------------+-------+-------+------+-------+-----+------------+
|                38313|   125|125|              125|           125|     749|       215|                 215|                 283|         221|    215|    304|   304|  23967|  304|         304|
+---------------------+------+---+-----------------+--------------+--------+----------+--------------------+--------------------+------------+-------+-------+------+-------+-----+------------+

